<a href="https://colab.research.google.com/github/csgm2328/Emily/blob/master/Verify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#예측한 모델 불러와서 현재시간에서 검증하고 새로운 예측하기

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr #data 받아오는 api 라이브러리
from pandas import json_normalize
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
import os
from keras import metrics
from keras.layers import *
from keras.models import *
from keras.utils.np_utils import to_categorical
from keras.callbacks import *
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
plt.style.use('bmh')

#**Load Model & Setting for Verify**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/LSTM_models/

In [ ]:
ls

In [ ]:
##Verify
from keras.models import load_model
model_name = '원익IPS: [2012-01-01 ~ 2020-09-22] 128cell 100win 1bat 10000ep 0.2drop True 0.0001lr.h5'
model = load_model(model_name)

In [ ]:
!pip install jamotools

In [ ]:
import jamotools #hangul_utils은 설치안되며 보다 나은 패키지
it = model_name.split(':') #이방법으로하면 모음,자음별로 삼성전자가 11자리. 데이터프레임에서 찾으려면 4글자로 자음모음 합쳐주기
item_name = jamotools.join_jamos(it[0])

In [ ]:
#1. price 개수를 검증하려는 일자에 맞춤.

train_start = model_name.split()[1].split('-')
s_y = train_start[0][1:]
s_m = train_start[1]
s_d = train_start[2]
start_date = datetime(int(s_y), int(s_m), int(s_d))

train_end = model_name.split()[3].split('-')
y = train_end[0]
m = train_end[1]
d = train_end[2][:-1]

verif_gap = datetime.date(datetime.today()) - datetime.date(datetime(int(y), int(m), int(d)))
verif_gap = verif_gap.days
  
predict_day = 10
n_epoch = model_name.split(' ')[7][:-2]
window_size = int(model_name.split(' ')[5][:-3])
n_batch = int(model_name.split(' ')[6][:-3])
Title = model_name + ' [' + str(datetime.date(datetime.today())) + ' Verified]'
print(Title,verif_gap,window_size,n_batch, sep='\n')

#Request API

In [ ]:
#1405699200 2014.7.18 01:00(GMT+9)부터 24시간 주기
#milliseconds 단위로 변환시켜주는 timestamp()
#start_date = datetime(2017,5,1)
start = int(start_date.timestamp()) 
ret = requests.get('https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start={}&end=9999999999&period=86400'.format(start))
print(ret)

In [ ]:
js = ret.json()
print(js[-1])
df = pd.DataFrame(js)

In [ ]:
scaler = MinMaxScaler()
real_price = df['close'].tolist()
price = scaler.fit_transform(df[['close']]).reshape(-1).tolist()  

print('오늘은',datetime.today(), real_price[-1], len(real_price), '개') # GMT +0 기준

In [ ]:
stock_url = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
stock_url.종목코드 = stock_url.종목코드.map('{:06d}'.format)
stock_url = stock_url[['회사명', '종목코드']]
stock_url = stock_url.rename(columns={'회사명':'name', '종목코드':'code'})

In [ ]:
code = stock_url.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip()
dr_code = code + '.KQ' 
print(item_name, dr_code)

#DataReader crawling

In [ ]:
start = (2014, 1, 1)
start = datetime(*start)
end = datetime.today()
df = pdr.DataReader(dr_code, 'yahoo', start, end) 

In [ ]:
scaler = MinMaxScaler()
real_price = df['Close'].tolist()
price = scaler.fit_transform(df[['Close']]).reshape(-1).tolist()
print('오늘은',datetime.today(), real_price[-1], len(real_price), '개')

#make DataSet

In [ ]:
x = []  
y = []  

for i in range(len(price) - window_size):   
  x.append([price[i+j] for j in range(window_size)])
  y.append(price[window_size + i])

x = np.asarray(x)
y = np.asarray(y)
print(x.shape) 
print(type(y)) 

In [ ]:
train_test_split = int(len(x)*0.7) 
split_pt = train_test_split + window_size

x_train = x[:train_test_split, :]
y_train = y[:train_test_split]

x_test = x[train_test_split:, :]
y_test = y[train_test_split:]

xtrain = x_train.reshape((x_train.shape[0], x_train.shape[1], 1)) 
xtest =  np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1)) 

print(xtrain.shape) 
print(x_test.shape) 

#**Prediction**

In [ ]:
#train, test prediction
train_pred = model.predict(xtrain,batch_size=n_batch) # x[0~window_size-1] 만큼의 특징벡터를 학습해서 학습데이터 수 
model.reset_states()
test_pred = model.predict(xtest,batch_size=n_batch)
model.reset_states()
real_train_pred = scaler.inverse_transform(train_pred)
real_test_pred = scaler.inverse_transform(test_pred)

#(t+n)일 예측하기 

In [ ]:
#(t+n)일 예측 시뮬레이션 (순수 predict)
def predict_future():
  if (verif_gap != 0):
    today = xtest[-verif_gap-1]
    today = np.vstack((today[1:], price[-verif_gap]))
    seq_in = today 
    fut = np.zeros((verif_gap+predict_day,1))

    i = 0
    for i in range(verif_gap): #과거 예측결과 평가를 위한
      sample_in = seq_in.reshape(1,window_size,1)
      after = model.predict(sample_in, batch_size=n_batch)
      model.reset_states() #없으면 예측값 계속 바뀜
      seq_in = np.vstack((seq_in[1:], after))
      fut[i] = after
    
    #검증 후 예측
    today = xtest[-1]
    today = np.vstack((today[1:], price[-1]))
    seq_in = today

    i+=1 #배열 다음위치부터
    for i in range(i,i+predict_day,1):
      sample_in = seq_in.reshape(1,window_size,1)
      after = model.predict(sample_in, batch_size=n_batch)
      model.reset_states() 
      seq_in = np.vstack((seq_in[1:], after))
      fut[i] = after

  else: #그날 바로 검증일때
    today = xtest[-1]
    today = np.vstack((today[1:], price[-1]))
    seq_in = today 
    fut = np.zeros((predict_day,1))

    for i in range(predict_day): 
      sample_in = seq_in.reshape(1,window_size,1)
      after = model.predict(sample_in, batch_size=n_batch)
      model.reset_states() 
      seq_in = np.vstack((seq_in[1:], after))
      fut[i] = after
  

  real_fut = scaler.inverse_transform(fut)
  return real_fut, fut

In [ ]:
############Verify or Modify Simulation

#하루마다 갱신하는 예측 오늘치는 갱신하고 그 다음날거랑 마지막 거 새로예측
#하루늦게 추종하니까 안좋은 결과
verif_price = price
real_verif_price = real_price

today = xtest[-verif_gap]
today = np.vstack((today[1:], price[-verif_gap]))
seq_in = today #오늘꺼 까지 넣고 
after_pred = np.zeros((predict_day+verif_gap,1)) #15일치
sample_in = seq_in.reshape(1,window_size,1)
after = model.predict(sample_in, batch_size=n_batch)
model.reset_states() #없으면 예측값 계속 바뀜
after_pred[0] = after #첫번째꺼는 이미지났으니까 유지하고

#seq_in[window_size-predict_day-1] = verif_price[-verif_gap] # 7월 10일 기준. 11일 예측치를, 11일 actual data로 갱신
j = 1
for i in range(verif_gap, 0, -1): #
  seq_in = np.vstack((seq_in[1:], verif_price[-i])) #11일 acutual 추가
  sample_in = seq_in.reshape(1, window_size, 1)
  after = model.predict(sample_in, batch_size=n_batch) #갱신된 11일치로 12일치 다시 예측
  model.reset_states()
  #seq_in = np.vstack((seq_in[1:], after))    #12일 예측치를 after에만 추가 seq_in에는 actual만 들어가야하므로
  #after_pred = np.vstack((after_pred, after)) #전부다 갱신해줘야함 새로운 배열
  after_pred[j] = after
  j+=1

seq_in = np.vstack((seq_in[1:], after_pred[j-1])) # 17일을 위한 16일은 넣어주고
for i in range(j,predict_day+verif_gap):
  sample_in = seq_in.reshape(1, window_size, 1)
  after = model.predict(sample_in, batch_size=n_batch)
  model.reset_states()
  seq_in = np.vstack((seq_in[1:], after))
  after_pred[i] = after

real_after_pred = scaler.inverse_transform(after_pred)

In [ ]:
#MCC를 사용하는 이유
TP = 5
FN = 15 
TN = 5
FP = 0
#ACC는 91%지만, MCC는 균형적이지 못해서 낮게 나온다!! 하락 5번중 한번밖에 못맞췄으므로
MCC = (TP * TN - FP * FN) / np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
FM = np.sqrt((TP/(TP+FP) * (TN/(TP+FN))))
#print('%.2f%%' %(100*(0.5 + MCC/2)))
MCC

# *Matthews correlation coefficient*

In [ ]:
#F1 score는 지나치게 긍정적으로 평가되는 경향이 있다. 분류문제인 양수인지 음수인지를 판별할 때는 MCC를 사용한다
#MCC는 -1 ~ +1 사이 값을 나타내며 0은 임의 예측과 같은성능
#방향 정확성 평가, MCC는 상승하락에 대해 균형있게 예측했는지 평가
#8/27 수많은 모델 검사결과 MCC가 ACC보다 오히려 과긍정으로 나타났다. TP*TN - FP*FN 의 한계 때문이였음
def MCC(actual, target, t): 
  TP, TN, FP, FN = 0,0,0,0
  ACC_List = [] 
  MCC_List = []

  for i in range(len(target)):
    if (actual[i+t] - actual[i+t-1] > 0): #상승
      if (target[i] - actual[i+t-1] > 0): #상승예측
        TP += 1
      else:
        FN += 1
    elif (actual[i+t] - actual[i+t-1] < 0): # 하락
      if (target[i] - actual[i+t-1] < 0):
        TN += 1
      else:
        FP += 1
    else: #그대로
      if (target[i] == actual[i+t-1]):
        TP += 1
      elif (target[i] - actual[i+t-1] > 0):
        FP += 1
      else:
        FN += 1
    N = TP + TN + FP + FN
    S = (TP + FN) / N
    P = (TP + FP) / N
    ACC = (TP + TN) / N
    MCC = (TP / N - S * P) / np.sqrt(P*S*(1-S)*(1-P))
    ACC_List.append(ACC)
    MCC_List.append(MCC)

  N = TP + TN + FP + FN
  S = (TP + FN) / N
  P = (TP + FP) / N
  ACC = (TP + TN) / N
  MCC = (TP / N - S * P) / np.sqrt(P*S*(1-S)*(1-P))
  #MCC = (TP * TN - FP * FN) / np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
  #두개가 0이되면 측정 안된다
  print(TP,FN,TN,FP)
  print('방향정확도: %.3f%%\nMCC: %.2f\n' %(ACC*100,MCC))

In [ ]:
#keras 라이브러리 내에서 mape 평가는 값이 너무 큼 mae,mse 전부다
def MAPE(actual, target, t): #Mean absolute percentage error
  scores = []
  for i in range(len(target)):
    scores.append(np.abs((actual[i+t] - target[i]) / actual[i+t]))
  print('값 오차율: %.2f%%' %(np.average(scores)*100))
  #return scores

In [ ]:
real_after_pred, after_pred = predict_future()

In [ ]:
#정규화상태로 하면 0에 가까워질때 값이 너무커지는 함정
MAPE(real_price, real_train_pred, window_size) #list,dic,set 같은 mutable object는 call by reference임
MCC(real_price, real_train_pred, window_size) 
MAPE(real_price, real_test_pred, split_pt)
MCC(real_price, real_test_pred, split_pt)
if (verif_gap != 0 ):
  MAPE(real_price, real_after_pred[:verif_gap], split_pt + len(test_pred)-verif_gap)
  MCC(real_price, real_after_pred[:verif_gap], split_pt + len(test_pred)-verif_gap)
else :
  verif_date = 21
  MAPE(real_price, real_test_pred[:verif_date], split_pt + len(test_pred)-verif_date)
  MCC(real_price, real_test_pred[:verif_date], split_pt + len(test_pred)-verif_date)

In [ ]:
#완전적합부분 검사
MAPE(price, train_pred[190-window_size:562-window_size], 190)
MCC(price, train_pred[190-window_size:562-window_size], 190)

In [ ]:
trainScore = model.evaluate(xtrain, y_train, batch_size=n_batch, verbose=0)
testScore = model.evaluate(xtest, y_test, batch_size=n_batch, verbose=0)
print("mae: (%.5f, %.5f)" %(trainScore[1], testScore[1]))

#**Result**

In [ ]:
trace = go.Scatter(x=np.arange(0,len(price),1), y=price, #0~1915까지 해야함 재정규화로 matrix되면 reshape해주어 1차행렬로 만들어야함
mode = 'lines', name='actual')
trace2 = go.Scatter(x=np.arange(window_size,split_pt,1), y=train_pred.reshape(train_test_split), #(5,1205,1) 이지만, 5~1204까지 적용 다 마찬가지
mode = 'lines', name='train')
trace3 = go.Scatter(x=np.arange(split_pt,split_pt + len(test_pred),1), y=test_pred.reshape(len(test_pred)),  #1205~1915
mode = 'lines', name='prediction')
trace4 = go.Scatter(x=np.arange(split_pt +len(test_pred)-verif_gap, split_pt + len(test_pred) + len(after_pred), 1), y =after_pred.reshape(len(after_pred)),
mode = 'lines', name= 'after_pred')

data = [trace, trace2, trace3, trace4]
layout = go.Layout(title=Title)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig) 

from google.colab import files
import os

In [ ]:
#알고리즘
#골든 포인트와 데드 포인트 찾기
#첫번째 GP: 0일 얼마 DP: 3일 얼마 수익률 5%
af = after_pred
#1번. 보유중인가요?
holding = False

#2. 매수 시작
today = 0
GPs = [] #포인트들의 날짜를 저장할 예정
DPs = []
Points = []
print(af)
def DP_check(io):
  #다음날 부터 DP check
  DP_can = io
  while io < len(af)-1:
    print('io: %d' %io)
    today_profit = (af[io+1] - af[DP_can]) / af[DP_can] # 고점 기준 누적 손익률
    print(today_profit)
    if today_profit > 0: #새로운 고점 = DP 후보
      DP_can = io+1
      io+=1

    elif (af[io+1] - af[io]) / af[io] <= -0.02: #한번에 2%하락하면
      Points.append(DP_can)
      break;
    elif today_profit <= -0.03: #고점 대비 3%하락하면
      Points.append(DP_can) # DP on
      break;
    else:
      io+=1
  return DP_can+1 # GP, DP 추가후 새로운 기준  



if holding: #팔고 사는 데드포인트
  point_check()

else:#4. 그럼 골든포인트 찾기
  GP_can = 0
  t = 0
  while t < len(af)-1:
    
    today_profit = (af[t+1] - af[GP_can]) / af[GP_can] # 저점 기준 누적 손익률
    if today_profit < 0:
      GP_can = t+1 #새로운 저점 = GP 후보
      t+=1
    elif (af[t+1] - af[t]) / af[t] >= 0.03: #한번에 3% 상승시
      Points.append(GP_can)
      t = DP_check(GP_can+1)
      GP_can = t
      
    elif today_profit >= 0.05: #고점 대비 5% 상승하면
      Points.append(GP_can) #GP on, 매매 순서대로 넣기 
      t = DP_check(GP_can+1)
      GP_can = t

    else:
      t+=1

#결과 출력
from datetime import timedelta
buy = 0 #사는게 먼저일떄
print('최적 매매 타이밍은 아래와 같습니다.\n')

for i in Points:
  v = datetime.date(datetime(int(y), int(m), int(d))) + timedelta(days=i)
  p = real_after_pred.astype(int)[i]

  if buy == 0: 
    print('buy: %s -' %v.strftime('%Y-%m-%d'), "{:,}원".format(p[0])) #p는 array형식이라 안됨
    buy = 1
  else:
    print('sell: %s -' %v.strftime('%Y-%m-%d'), "{:,}원".format(p[0]))
    buy = 0
    


In [ ]:
plt.figure(figsize=(5,5))
plt.plot(np.arange(0,verif_gap,1), real_verif_price[split_pt+len(test_pred)-verif_gap:], color ='blue',label = 'actual')
plt.plot(np.arange(0,len(real_after_pred), 1), real_after_pred, color='red', label='predict')
plt.legend()